# Examine census response rates, with data at state and census tract levels
## 2020 response rates from: https://2020census.gov/en/response-rates.html
## 2010 response rates from: https://api.census.gov/data/2010/dec/responserate/variables.html
## Demographic information in 2014-2018 ACS 5-year-estimate from: https://data2.nhgis.org/main

In [1]:
import pandas as pd
import numpy as np

# in excel changed original dates from m/d/yyyy to yyyy-mm-dd
initial_df = pd.read_csv('decennialrr2020_working.csv')
# had to resave as UTF-8 CSV, hence the 2
crosswalk = pd.read_csv('decennialrr2020_crosswalkfile2.csv')
# states paired with region as defined by census map at https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
regions = pd.read_csv('state_region.csv')

## Columns


GEO_ID = Geographic Identifier

RESP_DATE = Posting Date

State = name of state (one of the 50 states, District of Columbia, Puerto Rico, or NaN)

Geo_Name = name of the tract, county, state

Region = region of the U.S. in which state is located as defined by census map at https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

Geo_Type = type of geography; possible answers include Census Tract, Congressional District, Consolidated City, Country, County, County Subdivision, Place, Region, State, Tribal Tract, Tribal Area

DRRINT = Daily Self-Response Rate - Internet

DRRALL = Daily Self-Response Rate – Overall

CRRINT = Cumulative Self-Response Rate - Internet

not_int = new calculated column showing response rate NOT from internet

CRRALL = Cumulative Self-Response Rate – Overall

DINTMIN = Minimum Daily Internet Self-Response Rate

DMIN = Minimum Daily Overall Self-Response Rate

CINTMIN = Minimum Cumulative Internet Self-Response Rate

CMIN = Minimum Cumulative Overall Self-Response Rate

DINTMAX = Maximum Daily Internet Self-Response Rate

DMAX = Maximum Daily Overall Self-Response Rate

CINTMAX = Maximum Cumulative Internet Self-Response Rate

CMAX = Maximum Cumulative Overall Self-Response Rate

DINTAVG = Average Daily Internet Self-Response Rate

DAVG = Average Daily Overall Self-Response Rate

CINTAVG = Average Cumulative Internet Self-Response Rate

CAVG = Average Cumulative Overall Self-Response Rate

DINTMED = Median Daily Internet Self-Response Rate

DMED = Median Daily Overall Self-Response Rate

CINTMED = Median Cumulative Internet Self-Response Rate

CMED = Median Cumulative Overall Self-Response Rate

In [2]:
# merge responses and crosswalk
merged1 = pd.merge(initial_df, crosswalk, on='GEO_ID')

#merge merged1 with region data
merged = pd.merge(merged1, regions, on='State')

# create column showing responses not from internet
merged['not_int'] = merged.CRRALL - merged.CRRINT

#reorder columns to move State, Geo_Name and Geo_Type to front; also going to drop some values
cols = merged.columns.tolist()
cols = ['GEO_ID', 'RESP_DATE', 'State', 'Geo_Name', 'Region', 'Geo_Type', 
        'CRRINT', 'not_int', 'CRRALL']
merged = merged[cols]
merged = merged.rename(columns={'CRRINT':'internet', 'CRRALL':'2020_rate'})
merged

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,internet,not_int,2020_rate
0,0100000US,6/15/2020,NaN,United States,na,Country,49.1,12.3,61.4
1,0200000US1,6/15/2020,NaN,Northeast,na,Region,49.1,11.7,60.8
2,0200000US2,6/15/2020,NaN,Midwest,na,Region,52.7,13.8,66.5
3,0200000US3,6/15/2020,NaN,South,na,Region,45.4,13.2,58.6
4,0200000US4,6/15/2020,NaN,West,na,Region,52.0,9.9,61.9
...,...,...,...,...,...,...,...,...,...
123245,2560000US4755T00100,6/15/2020,Tribal Land,"T001, Ysleta del Sur",Tribal Land,Tribal Tract,32.7,3.1,35.8
123246,2560000US4760T00100,6/15/2020,Tribal Land,"T001, Yurok",Tribal Land,Tribal Tract,10.7,0.2,10.9
123247,2560000US4770T00100,6/15/2020,Tribal Land,"T001, Zia",Tribal Land,Tribal Tract,17.4,0.4,17.8
123248,2560000US4785T00100,6/15/2020,Tribal Land,"T001, Zuni",Tribal Land,Tribal Tract,8.0,0.1,8.1


## Census Tracts

In [3]:
# select just census tract geo types
tracts = merged[merged['Geo_Type'].str.contains("Tract")]

# sort by highest cumulative response rate
tracts.sort_values(by='2020_rate', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,internet,not_int,2020_rate
26459,1400000US13215010606,6/15/2020,Georgia,"Tract 106.06, Muscogee",South,Census Tract,0.0,98.1,98.1
49682,1400000US25013812903,6/15/2020,Massachusetts,"Tract 8129.03, Hampden",Midwest,Census Tract,86.2,6.9,93.1
53285,1400000US26099223801,6/15/2020,Michigan,"Tract 2238.01, Macomb",Midwest,Census Tract,86.2,6.6,92.8
113068,1400000US51059492202,6/15/2020,Virginia,"Tract 4922.02, Fairfax",South,Census Tract,89.1,3.3,92.4
54097,1400000US26139021605,6/15/2020,Michigan,"Tract 216.05, Ottawa",Midwest,Census Tract,85.4,7.0,92.4
...,...,...,...,...,...,...,...,...,...
3401,1400000US04017940008,6/15/2020,Arizona,"Tract 9400.08, Navajo",West,Census Tract,0.0,0.0,0.0
122937,2560000US1440T00100,6/15/2020,Tribal Land,"T001, Havasupai",Tribal Land,Tribal Tract,0.0,0.0,0.0
73127,1400000US36047005303,6/15/2020,New York,"Tract 53.03, Kings",Northeast,Census Tract,0.0,0.0,0.0
7920,1400000US06037277400,6/15/2020,California,"Tract 2774, Los Angeles",West,Census Tract,0.0,0.0,0.0


#### Tribal tracts

In [48]:
# create df with response rates in tribal tracts
tribal = tracts[tracts['Geo_Type'].str.contains("Tribal")]
tribal.sort_values(by='2020_rate', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,internet,not_int,2020_rate
123070,2560000US2560T00400,6/15/2020,Tribal Land,"T004, Oneida (WI)",Tribal Land,Tribal Tract,78.8,6.6,85.4
122956,2560000US1610T00600,6/15/2020,Tribal Land,"T006, Isabella",Tribal Land,Tribal Tract,69.1,10.8,79.9
123069,2560000US2560T00300,6/15/2020,Tribal Land,"T003, Oneida (WI)",Tribal Land,Tribal Tract,68.4,9.9,78.3
123108,2560000US3000T00200,6/15/2020,Tribal Land,"T002, Puyallup",Tribal Land,Tribal Tract,71.4,6.1,77.5
123068,2560000US2560T00200,6/15/2020,Tribal Land,"T002, Oneida (WI)",Tribal Land,Tribal Tract,67.3,9.5,76.8
123101,2560000US2910T00100,6/15/2020,Tribal Land,"T001, Port Gamble",Tribal Land,Tribal Tract,72.6,3.9,76.5
123067,2560000US2560T00100,6/15/2020,Tribal Land,"T001, Oneida (WI)",Tribal Land,Tribal Tract,68.6,6.9,75.5
123113,2560000US3000T00800,6/15/2020,Tribal Land,"T008, Puyallup",Tribal Land,Tribal Tract,69.2,6.2,75.4
123071,2560000US2560T00500,6/15/2020,Tribal Land,"T005, Oneida (WI)",Tribal Land,Tribal Tract,62.5,12.0,74.5
122879,2560000US0750T00100,6/15/2020,Tribal Land,"T001, Colusa",Tribal Land,Tribal Tract,73.3,0.0,73.3


In [5]:
### tribal areas and tracts stats

#mean non internet response
tribal.mean(axis=0)['not_int']

7.720422535211265

In [6]:
# mean internet response rate
tribal.mean(axis=0)['internet']

20.193896713615032

In [7]:
# mean overall response rate
tribal.mean(axis=0)['2020_rate']

27.914319248826278

#### Tracts with 0 overall response rate

In [47]:
## Tracts with 0 cumulative response rate: 28
is_zero = tracts['2020_rate'] == 0.0
zeros = tracts[is_zero]
zeros.sort_values(by='State')

## Tracts with 0 cumulative response rate: 28

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,internet,not_int,2020_rate
2305,1400000US04005980000,6/15/2020,Arizona,"Tract 9800, Coconino",West,Census Tract,0.0,0.0,0.0
3401,1400000US04017940008,6/15/2020,Arizona,"Tract 9400.08, Navajo",West,Census Tract,0.0,0.0,0.0
7920,1400000US06037277400,6/15/2020,California,"Tract 2774, Los Angeles",West,Census Tract,0.0,0.0,0.0
9345,1400000US06037980020,6/15/2020,California,"Tract 9800.20, Los Angeles",West,Census Tract,0.0,0.0,0.0
13265,1400000US06083980100,6/15/2020,California,"Tract 9801, Santa Barbara",West,Census Tract,0.0,0.0,0.0
17975,1400000US10001980000,6/15/2020,Delaware,"Tract 9800, Kent",South,Census Tract,0.0,0.0,0.0
18595,1400000US12005000700,6/15/2020,Florida,"Tract 7, Bay",South,Census Tract,0.0,0.0,0.0
46001,1400000US22125951703,6/15/2020,Louisiana,"Tract 9517.03, West Feliciana",South,Census Tract,0.0,0.0,0.0
53125,1400000US26083980100,6/15/2020,Michigan,"Tract 9801, Keweenaw",Midwest,Census Tract,0.0,0.0,0.0
53490,1400000US26099982100,6/15/2020,Michigan,"Tract 9821, Macomb",Midwest,Census Tract,0.0,0.0,0.0


## States

In [9]:
# create df with response rate by state
is_states = merged['Geo_Type'] == 'State'
states = merged[is_states]
states = states.rename(columns={"internet": "state_internet", "not_int" : "state_not_int", "2020_rate": "2020_state_rate"})

# print df and sort by highest cumulative response rate
states.sort_values(by='2020_state_rate', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,state_internet,state_not_int,2020_state_rate
55595,0400000US27,6/15/2020,Minnesota,Minnesota,Midwest,State,59.7,11.2,70.9
117774,0400000US55,6/15/2020,Wisconsin,Wisconsin,Midwest,State,56.4,12.0,68.4
50831,0400000US26,6/15/2020,Michigan,Michigan,Midwest,State,53.0,14.7,67.7
38079,0400000US19,6/15/2020,Iowa,Iowa,Midwest,State,53.3,14.4,67.7
64533,0400000US31,6/15/2020,Nebraska,Nebraska,Midwest,State,53.6,14.0,67.6
114850,0400000US53,6/15/2020,Washington,Washington,West,State,57.9,8.9,66.8
112324,0400000US51,6/15/2020,Virginia,Virginia,South,State,54.3,12.0,66.3
28633,0400000US17,6/15/2020,Illinois,Illinois,Midwest,State,54.0,12.1,66.1
82562,0400000US39,6/15/2020,Ohio,Ohio,Midwest,State,51.0,15.1,66.1
34724,0400000US18,6/15/2020,Indiana,Indiana,Midwest,State,50.6,15.3,65.9


In [10]:
# highest non-internet response rate (not_int)
states.sort_values(by='state_not_int', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,state_internet,state_not_int,2020_state_rate
59796,0400000US28,6/15/2020,Mississippi,Mississippi,South,State,34.2,22.4,56.6
4047,0400000US05,6/15/2020,Arkansas,Arkansas,South,State,37.4,18.9,56.3
116938,0400000US54,6/15/2020,West Virginia,West Virginia,South,State,34.8,18.2,53.0
5,0400000US01,6/15/2020,Alabama,Alabama,South,State,41.1,18.1,59.2
42735,0400000US21,6/15/2020,Kentucky,Kentucky,South,State,47.9,17.1,65.0
44576,0400000US22,6/15/2020,Louisiana,Louisiana,South,State,39.9,15.8,55.7
100434,0400000US47,6/15/2020,Tennessee,Tennessee,South,State,45.2,15.8,61.0
34724,0400000US18,6/15/2020,Indiana,Indiana,Midwest,State,50.6,15.3,65.9
82562,0400000US39,6/15/2020,Ohio,Ohio,Midwest,State,51.0,15.1,66.1
88050,0400000US40,6/15/2020,Oklahoma,Oklahoma,South,State,40.9,14.9,55.8


#### State stats

In [11]:
states.mean(axis=0)['state_not_int']

12.49807692307692

In [12]:
states.mean(axis=0)['state_internet']

47.42692307692311

In [13]:
states.mean(axis=0)['2020_state_rate']

59.92500000000002

In [14]:
# average by region
# NOTE as tribal tracts are not assigned to a state they do not have a corresponding region and thus are not counted in the regional calculations
states.groupby('Region').mean().sort_values(by='2020_state_rate', ascending=False)

,state_internet,state_not_int,2020_state_rate
Region,,,
Midwest,52.430769,13.292308,65.723077
Northeast,47.662500,11.887500,59.550000
West,49.115385,10.092308,59.207692
South,44.370588,14.258824,58.629412
Puerto Rico,10.500000,8.400000,18.900000


#### State stats without Puerto Rico

In [15]:
no_pr = states[states.State != 'Puerto Rico']

# average non internet response rate
no_pr.mean(axis=0)['state_not_int']

12.578431372549018

In [16]:
# average internet response
no_pr.mean(axis=0)['state_internet']

48.150980392156896

In [17]:
# average overall response rate
no_pr.mean(axis=0)['2020_state_rate']

60.729411764705894

In [18]:
# average region response rate (so south now excludes Puerto Rico)
no_pr.groupby('Region').mean().sort_values(by='2020_state_rate', ascending=False)

,state_internet,state_not_int,2020_state_rate
Region,,,
Midwest,52.430769,13.292308,65.723077
Northeast,47.662500,11.887500,59.550000
West,49.115385,10.092308,59.207692
South,44.370588,14.258824,58.629412


## Compare to 2010 response rates

### 2010 to 2020 State Response Differences

In [19]:
# read in csvs with 2010 response data for tracts and states
rates2010 = pd.read_csv('2010responserate.csv')
#rename this column
rates2010 = rates2010.rename(columns={'FSRR2010':'2010_rate'})
rates2010

,NAME,county,State,GEO_ID,2010_rate,state_num,county_num,tract
0,Census Tract 201,Autauga County,Alabama,1400000US01001020100,70.6,1,1,20100
1,Census Tract 202,Autauga County,Alabama,1400000US01001020200,70.1,1,1,20200
2,Census Tract 203,Autauga County,Alabama,1400000US01001020300,73.6,1,1,20300
3,Census Tract 204,Autauga County,Alabama,1400000US01001020400,78.4,1,1,20400
4,Census Tract 205.01,Autauga County,Alabama,1400000US01001020501,81.2,1,1,20501
...,...,...,...,...,...,...,...,...
84514,Census Tract 7505.01,Yauco Municipio,Puerto Rico,1400000US72153750501,70.4,72,153,750501
84515,Census Tract 7505.02,Yauco Municipio,Puerto Rico,1400000US72153750502,73.4,72,153,750502
84516,Census Tract 7505.03,Yauco Municipio,Puerto Rico,1400000US72153750503,63.4,72,153,750503
84517,Census Tract 7506.01,Yauco Municipio,Puerto Rico,1400000US72153750601,67.1,72,153,750601


In [20]:
## difference in row numbers: both tract dfs have 84519 rows, but when joined only 84093
# Identify what values are in rates2010 and not in tracts
key_diff1 = set(rates2010.GEO_ID).difference(tracts.GEO_ID)
len(key_diff1)
key_diff1

# Identify what values are in tracts and not in rates2010
key_diff2 = set(tracts.GEO_ID).difference(rates2010.GEO_ID)
len(key_diff2)
key_diff2

{'2560000US0010T00100',
 '2560000US0020T00100',
 '2560000US0020T00200',
 '2560000US0020T00300',
 '2560000US0020T00400',
 '2560000US0020T00500',
 '2560000US0020T00600',
 '2560000US0020T00700',
 '2560000US0020T00800',
 '2560000US0020T00900',
 '2560000US0020T01000',
 '2560000US0050T00100',
 '2560000US0080T00100',
 '2560000US0080T00200',
 '2560000US0110T00100',
 '2560000US0115T00100',
 '2560000US0140T00100',
 '2560000US0155T00100',
 '2560000US0165T00100',
 '2560000US0170T00100',
 '2560000US0185T00100',
 '2560000US0200T00100',
 '2560000US0225T00100',
 '2560000US0250T00100',
 '2560000US0265T00100',
 '2560000US0275T00100',
 '2560000US0290T00100',
 '2560000US0305T00100',
 '2560000US0305T00200',
 '2560000US0305T00300',
 '2560000US0325T00100',
 '2560000US0335T00100',
 '2560000US0350T00100',
 '2560000US0360T00100',
 '2560000US0400T00100',
 '2560000US0415T00100',
 '2560000US0435T00100',
 '2560000US0440T00100',
 '2560000US0450T00100',
 '2560000US0510T00100',
 '2560000US0525T00100',
 '2560000US0540T

##### Conclusion: 2010 rates do not include tribal tracts while 2020 tracts are missing some tracts in a multitude of states, likely due to a change in tract boundaries. Those differences account for 426 tracts, which is .5% of the original 84519 tracts. As these tracts are small percentage of all tracts, they can be dropped. 

In [21]:
# merge with 2020 tracts
tracts_merged = pd.merge(tracts, rates2010, on='GEO_ID')

#get column names
cols = tracts_merged.columns.tolist()
#select only columns we want
cols = ['GEO_ID','Geo_Name','county', 'State_y', 'Region', '2020_rate', '2010_rate']
tracts_merged = tracts_merged[cols]
#rename weird column name
tracts_merged = tracts_merged.rename(columns={'State_y':'State'})
#print df sorted largest --> smallest 2010 rate
tracts_merged.sort_values(by='2010_rate', ascending=False)

,GEO_ID,Geo_Name,county,State,Region,2020_rate,2010_rate
10379,1400000US06071010906,"Tract 109.06, San Bernardino",San Bernardino County,California,West,10.8,100.0
48849,1400000US35035940000,"Tract 9400, Otero",Otero County,New Mexico,West,15.9,100.0
13467,1400000US08005980000,"Tract 9800, Arapahoe",Arapahoe County,Colorado,West,87.5,100.0
8287,1400000US06053980000,"Tract 9800, Monterey",Monterey County,California,West,0.7,100.0
6449,1400000US06037265301,"Tract 2653.01, Los Angeles",Los Angeles County,California,West,4.8,100.0
...,...,...,...,...,...,...,...
83055,1400000US56013940202,"Tract 9402.02, Fremont",Fremont County,Wyoming,West,19.5,NaN
83056,1400000US56013940301,"Tract 9403.01, Fremont",Fremont County,Wyoming,West,27.8,NaN
83057,1400000US56013940302,"Tract 9403.02, Fremont",Fremont County,Wyoming,West,36.3,NaN
83058,1400000US56013940400,"Tract 9404, Fremont",Fremont County,Wyoming,West,46.5,NaN


In [22]:
#how many null 2010 response values are there: 531, which is .6% of all rows, 84093
is_no_2010 = tracts_merged.isnull()
no_2010 = is_no_2010.any(axis=1)
no_2010 = tracts_merged[no_2010]
no_2010.sort_values(by="2020_rate")

#due to the low percentage, these null values will be discarded
tracts_merged = tracts_merged.dropna(axis=0)
#check they'd discraded
tracts_merged.sort_values(by='2010_rate')
#there are no 2010 na values, so all were dropped

,GEO_ID,Geo_Name,county,State,Region,2020_rate,2010_rate
77029,1400000US50027966300,"Tract 9663, Windsor",Windsor County,Vermont,Northeast,20.4,0.0
48861,1400000US35039940800,"Tract 9408, Rio Arriba",Rio Arriba County,New Mexico,West,22.2,0.0
6111,1400000US06037207400,"Tract 2074, Los Angeles",Los Angeles County,California,West,25.9,0.0
21943,1400000US13089023115,"Tract 231.15, DeKalb",DeKalb County,Georgia,South,10.0,0.0
63439,1400000US42027012200,"Tract 122, Centre",Centre County,Pennsylvania,Northeast,17.6,0.0
...,...,...,...,...,...,...,...
13467,1400000US08005980000,"Tract 9800, Arapahoe",Arapahoe County,Colorado,West,87.5,100.0
24218,1400000US16001000703,"Tract 7.03, Ada",Ada County,Idaho,West,77.8,100.0
73376,1400000US48215020207,"Tract 202.07, Hidalgo",Hidalgo County,Texas,South,36.5,100.0
50116,1400000US36035970400,"Tract 9704, Fulton",Fulton County,New York,Northeast,19.6,100.0


In [23]:
#create column with difference in 2010 vs 2020 response rate
tracts_merged['10-20 difference'] = tracts_merged['2010_rate'] - tracts_merged['2020_rate']
#sort df largest --> smallest 10-20 difference
tracts_merged.sort_values(by='10-20 difference', ascending=False)

,GEO_ID,Geo_Name,county,State,Region,2020_rate,2010_rate,10-20 difference
8287,1400000US06053980000,"Tract 9800, Monterey",Monterey County,California,West,0.7,100.0,99.3
7980,1400000US06037980021,"Tract 9800.21, Los Angeles",Los Angeles County,California,West,1.6,100.0,98.4
49059,1400000US36001000404,"Tract 4.04, Albany",Albany County,New York,Northeast,1.8,100.0,98.2
53795,1400000US36103159510,"Tract 1595.10, Suffolk",Suffolk County,New York,Northeast,2.2,100.0,97.8
6449,1400000US06037265301,"Tract 2653.01, Los Angeles",Los Angeles County,California,West,4.8,100.0,95.2
...,...,...,...,...,...,...,...,...
21302,1400000US13051010707,"Tract 107.07, Chatham",Chatham County,Georgia,South,68.1,2.4,-65.7
40288,1400000US26163982400,"Tract 9824, Wayne",Wayne County,Michigan,Midwest,83.1,16.7,-66.4
65389,1400000US42101036400,"Tract 364, Philadelphia",Philadelphia County,Pennsylvania,Northeast,80.2,13.5,-66.7
22281,1400000US13121003700,"Tract 37, Fulton",Fulton County,Georgia,South,67.3,0.0,-67.3


In [24]:
# average difference as of 6/15/20
#this can take a while to run so is commented out unless needed
#tracts_merged.mean(axis=0)['10-20 difference']

In [25]:
# average difference by region
tracts_merged.groupby('Region').mean().sort_values(by='10-20 difference', ascending=False)

,2020_rate,2010_rate,10-20 difference
Region,,,
Puerto Rico,18.941477,56.631705,37.690228
South,58.490032,66.022076,7.532044
Northeast,60.211734,67.728176,7.516442
West,62.344020,67.693454,5.349433
Midwest,65.923700,70.971303,5.047603


### 2010 to 2020 State Response Differences

In [26]:
# read in csvs with 2010 response data for states
states2010 = pd.read_csv('states2010.csv')

# merge with 2020 states
states_merged = pd.merge(states, states2010, on='State')
#get the column names
cols = states_merged.columns.tolist()
#only select columns we want
cols = ['GEO_ID', 'State', 'Region',
 '2020_state_rate', '2000_rate', '2010_rate']
states_merged = states_merged[cols]

#create column with difference in 2010 vs 2020 response rate
states_merged['10-20 difference'] = states_merged['2010_rate'] - states_merged['2020_state_rate']

#print table sorted by 10-20 difference largest ---> smallest
states_merged.sort_values(by='10-20 difference', ascending=False)

,GEO_ID,State,Region,2020_state_rate,2000_rate,2010_rate,10-20 difference
51,0400000US72,Puerto Rico,Puerto Rico,18.9,54,54,35.1
40,0400000US45,South Carolina,South,55.9,68,75,19.1
33,0400000US37,North Carolina,South,57.7,69,76,18.3
1,0400000US02,Alaska,West,46.8,67,64,17.2
9,0400000US12,Florida,South,58.6,71,74,15.4
19,0400000US23,Maine,Northeast,52.8,67,68,15.2
31,0400000US35,New Mexico,West,49.8,68,65,15.2
42,0400000US47,Tennessee,South,61.0,72,76,15.0
43,0400000US48,Texas,South,56.1,70,71,14.9
45,0400000US50,Vermont,Northeast,54.3,68,69,14.7


In [27]:
# average difference as of 6/9/20
states_merged.mean(axis=0)['10-20 difference']

12.901923076923076

In [28]:
# average difference by region
states_merged.groupby('Region').mean().sort_values(by='10-20 difference', ascending=False)

,2020_state_rate,2000_rate,2010_rate,10-20 difference
Region,,,,
Puerto Rico,18.900000,54.000000,54.000000,35.100000
South,58.629412,70.647059,72.176471,13.547059
Northeast,59.550000,72.875000,72.375000,12.825000
West,59.207692,72.538462,70.923077,11.715385
Midwest,65.723077,79.230769,77.307692,11.584615


## Demographic Data

In [50]:
#load both sets of demographic data, join
demo1 = pd.read_csv('demographics1_working.csv')
demo2 = pd.read_csv('demographics2_working.csv')
demo = pd.merge(demo1, demo2, on=['GISJOIN', "YEAR", "STATE", "STATEA", 
                                              'COUNTY', 'COUNTYA', 'TRACTA', 
                                              'Geo_Name', 'NAME_E'])
#view demographics table
demo

# merge to create new df with response rates and demos for all tracts
df = pd.merge(tracts_merged, demo, on='Geo_Name')

#create new column adding up pop with rent > 30% income (homelessness marker)
df['rent_30_more'] = (df['rent_30_34.9'] + df['rent_35_39.9'] 
+ df['rent_40_49.9'] + df['rent_50_over'])
# check to see if column created
pd.set_option('display.max_columns', 100)
df

,GEO_ID,Geo_Name,county,State,Region,2020_rate,2010_rate,10-20 difference,GISJOIN,YEAR,STATE,STATEA,COUNTY,COUNTYA,TRACTA,NAME_E,total_population,total_population_race,white_alone,black_alone,amerindian_alone,asian_alone,pacific_islander_alone,other_alone,two_or_more,two_more_including_other,two_more_excluding_other,total_education,no_school,some_school,diploma,ged,some_college,associate,bachelor,master,prof_school,doctorate,language_total,lang_english_only,lang_spanish,lang_spanish_limited_english,lang_other_indo_euro,lang_asian_pacific_island,lang_other,income_poverty_ratio,income_poverty_under_half,income_poverty_half_.99,income_povery_1_1.24,income_poverty_1.25_1.49,income_poverty_1.5_1.84,income_poverty_1.85_1.99,income_poverty_2_over,median_household_income,per_capita_income,employment_total,labor_force,civilian_labor_force,civilian_employed,civilian_unemployed,armed_forces,not_labor_force,total_houses,occupied_houses,vacant_houses,total_occupied_houses,owner_occupied,renter_occupied,median_gross_rent,rent_to_income,rent_less_10,rent_10_14.9,rent_15_19.9,rent_20_24.9,rent_25_29.9,rent_30_34.9,rent_35_39.9,rent_40_49.9,rent_50_over,rent_not_computed,total_computer_status,has_computer,dial_up_computer,broadband_computer,no_internet_computer,no_computer,us_pop,us_born,us_territory_born,us_born_abroad,us_naturalization,not_us_citizen,total_pr,pr_born,pr_us_born,pr_born_abroad,pr_naturalization,pr_not_us_citizen,rent_30_more
0,1400000US01001020100,"Tract 201, Autauga",Autauga County,Alabama,South,64.3,70.6,6.3,G0100010020100,2014-2018,Alabama,1,Autauga County,1,20100,"Tract 201, Autauga Alabama",1923,1923,1609,161,0,17,0,0,136,0,136,1303,0,123,346,97,225,102,236,138,21,15,765,708,40,14,12,5,0,1923.0,41.0,177.0,114.0,52.0,59.0,12.0,1468.0,58625.0,31580.0,1562.0,966.0,966.0,931.0,35.0,0.0,596.0,779,765,14,765,570,195,827.0,195.0,21.0,23.0,0.0,5.0,0.0,4.0,14.0,46.0,45.0,37.0,765,625,0,549,76,140,1923,1870,0,11,18,24,0,0,0,0,0,0,109.0
1,1400000US01001020200,"Tract 202, Autauga",Autauga County,Alabama,South,65.6,70.1,4.5,G0100010020200,2014-2018,Alabama,1,Autauga County,1,20200,"Tract 202, Autauga Alabama",2028,2028,819,1129,0,19,27,9,25,0,25,1443,11,234,499,87,283,87,182,48,5,7,719,699,0,0,16,4,0,1818.0,78.0,247.0,106.0,75.0,88.0,208.0,1016.0,43531.0,19376.0,1646.0,790.0,785.0,750.0,35.0,5.0,856.0,852,719,133,719,464,255,775.0,255.0,0.0,30.0,14.0,12.0,52.0,26.0,25.0,3.0,80.0,13.0,719,519,0,468,51,200,2028,1993,0,10,15,10,0,0,0,0,0,0,134.0
2,1400000US01001020300,"Tract 203, Autauga",Autauga County,Alabama,South,73.8,73.6,-0.2,G0100010020300,2014-2018,Alabama,1,Autauga County,1,20300,"Tract 203, Autauga Alabama",3476,3476,2306,730,0,30,5,299,106,0,106,2406,26,281,708,142,628,166,249,187,0,19,1296,1196,48,0,15,37,0,3476.0,198.0,325.0,164.0,138.0,79.0,402.0,2170.0,51875.0,22527.0,2704.0,1684.0,1679.0,1624.0,55.0,5.0,1020.0,1397,1296,101,1296,841,455,917.0,455.0,0.0,5.0,50.0,43.0,73.0,35.0,30.0,23.0,135.0,61.0,1296,1081,0,950,131,215,3476,3231,5,60,49,131,0,0,0,0,0,0,223.0
3,1400000US01001020400,"Tract 204, Autauga",Autauga County,Alabama,South,77.8,78.4,0.6,G0100010020400,2014-2018,Alabama,1,Autauga County,1,20400,"Tract 204, Autauga Alabama",3831,3831,3382,296,24,8,0,0,121,18,103,2769,14,151,744,175,508,272,602,177,74,52,1639,1578,31,16,13,9,8,3831.0,29.0,80.0,151.0,214.0,414.0,34.0,2909.0,54050.0,30527.0,3097.0,1752.0,1712.0,1663.0,49.0,40.0,1345.0,1867,1639,228,1639,1262,377,966.0,377.0,16.0,37.0,43.0,85.0,33.0,39.0,28.0,7.0,71.0,18.0,1639,1468,7,1315,146,171,3831,3720,0,30,30,51,0,0,0,0,0,0,145.0
4,1400000US01001020600,"Tract 206, Autauga",Autauga County,Alabama,South,66.2,71.9,5.7,G0100010020600,2014-2018,Alabama,1,Autauga County,1,20600,"Tract 206, Autauga Alabama",3705,3705,2726,878,0,0,0,57,44,0,44,2455,43,367,678,194,507,161,346,105,22,32,1332,1307,16,0,9,0,0,3689.0,100.0,696.0,330.0,187.0,308.0,38.0,2030.0,46688.0,28049.0,3004.0,1451.0,1443.0,1333.0,110.0,8.0,1553.0,1413,1332,81,1332,1031,301,732.0,301.0,7.0,79.0,54.0,8.0,1

In [51]:
#df.std()Returns the standard deviation of each column
#df.corr()Returns the correlation between columns in a data frame

#In order to get # of null/missing values for each column, run 
pd.set_option('display.max_rows', 100)
pd.isnull(df).sum()

# make a dataframe of all rows with na value
df1 = df[df.isna().any(axis=1)]
df1
# how many nas in each state
hm = df1['State'].value_counts()
hm = pd.DataFrame(hm)
hm = hm.reset_index().rename(columns={'index':'state', 'State':'count_na'})
hm
#compare to total 
hmm = df['State'].value_counts()
hmm = pd.DataFrame(hmm)
hmm = hmm.reset_index().rename(columns={'index':'state', 'State':'count'})
hmm

#see what percent of state values of na
#Puerto Rico will lose the greatest % if these are dropped
hmmm = pd.merge(hm, hmm, on=['state'])
hmmm['na_percent'] = (hmmm['count_na']*100) / hmmm['count']
hmmm

#how many total nas? --> 1171
sum(hmmm['count_na'])

#nas account for what total % of all rows --> 1.82
(sum(hmmm['count_na'])*100) / sum(hmmm['count'])

1.8222561117940896

In [31]:
#Less than 2% of total, so for now will drop those rows
# before df had 64413 rows
df = df.dropna()    #now has 63242 rows, 1171 difference (total # nas)

In [32]:
#Df column names for reference
#dflist = df.columns.tolist()
#dflist

#print df with no nas
#df.to_csv("all_rates_demos_merged.csv")

#create df with just puerto rico
is_pr = df['Region'] == 'Puerto Rico'
pr = df[is_pr]
pr

#df without puerto rico and normalized for regression
regdf = df[df['Region'] != 'Puerto Rico']
regdf

,GEO_ID,Geo_Name,county,State,Region,2020_rate,2010_rate,10-20 difference,GISJOIN,YEAR,STATE,STATEA,COUNTY,COUNTYA,TRACTA,NAME_E,total_population,total_population_race,white_alone,black_alone,amerindian_alone,asian_alone,pacific_islander_alone,other_alone,two_or_more,two_more_including_other,two_more_excluding_other,total_education,no_school,some_school,diploma,ged,some_college,associate,bachelor,master,prof_school,doctorate,language_total,lang_english_only,lang_spanish,lang_spanish_limited_english,lang_other_indo_euro,lang_asian_pacific_island,lang_other,income_poverty_ratio,income_poverty_under_half,income_poverty_half_.99,income_povery_1_1.24,income_poverty_1.25_1.49,income_poverty_1.5_1.84,income_poverty_1.85_1.99,income_poverty_2_over,median_household_income,per_capita_income,employment_total,labor_force,civilian_labor_force,civilian_employed,civilian_unemployed,armed_forces,not_labor_force,total_houses,occupied_houses,vacant_houses,total_occupied_houses,owner_occupied,renter_occupied,median_gross_rent,rent_to_income,rent_less_10,rent_10_14.9,rent_15_19.9,rent_20_24.9,rent_25_29.9,rent_30_34.9,rent_35_39.9,rent_40_49.9,rent_50_over,rent_not_computed,total_computer_status,has_computer,dial_up_computer,broadband_computer,no_internet_computer,no_computer,us_pop,us_born,us_territory_born,us_born_abroad,us_naturalization,not_us_citizen,total_pr,pr_born,pr_us_born,pr_born_abroad,pr_naturalization,pr_not_us_citizen,rent_30_more
0,1400000US01001020100,"Tract 201, Autauga",Autauga County,Alabama,South,64.3,70.6,6.3,G0100010020100,2014-2018,Alabama,1,Autauga County,1,20100,"Tract 201, Autauga Alabama",1923,1923,1609,161,0,17,0,0,136,0,136,1303,0,123,346,97,225,102,236,138,21,15,765,708,40,14,12,5,0,1923.0,41.0,177.0,114.0,52.0,59.0,12.0,1468.0,58625.0,31580.0,1562.0,966.0,966.0,931.0,35.0,0.0,596.0,779,765,14,765,570,195,827.0,195.0,21.0,23.0,0.0,5.0,0.0,4.0,14.0,46.0,45.0,37.0,765,625,0,549,76,140,1923,1870,0,11,18,24,0,0,0,0,0,0,109.0
1,1400000US01001020200,"Tract 202, Autauga",Autauga County,Alabama,South,65.6,70.1,4.5,G0100010020200,2014-2018,Alabama,1,Autauga County,1,20200,"Tract 202, Autauga Alabama",2028,2028,819,1129,0,19,27,9,25,0,25,1443,11,234,499,87,283,87,182,48,5,7,719,699,0,0,16,4,0,1818.0,78.0,247.0,106.0,75.0,88.0,208.0,1016.0,43531.0,19376.0,1646.0,790.0,785.0,750.0,35.0,5.0,856.0,852,719,133,719,464,255,775.0,255.0,0.0,30.0,14.0,12.0,52.0,26.0,25.0,3.0,80.0,13.0,719,519,0,468,51,200,2028,1993,0,10,15,10,0,0,0,0,0,0,134.0
2,1400000US01001020300,"Tract 203, Autauga",Autauga County,Alabama,South,73.8,73.6,-0.2,G0100010020300,2014-2018,Alabama,1,Autauga County,1,20300,"Tract 203, Autauga Alabama",3476,3476,2306,730,0,30,5,299,106,0,106,2406,26,281,708,142,628,166,249,187,0,19,1296,1196,48,0,15,37,0,3476.0,198.0,325.0,164.0,138.0,79.0,402.0,2170.0,51875.0,22527.0,2704.0,1684.0,1679.0,1624.0,55.0,5.0,1020.0,1397,1296,101,1296,841,455,917.0,455.0,0.0,5.0,50.0,43.0,73.0,35.0,30.0,23.0,135.0,61.0,1296,1081,0,950,131,215,3476,3231,5,60,49,131,0,0,0,0,0,0,223.0
3,1400000US01001020400,"Tract 204, Autauga",Autauga County,Alabama,South,77.8,78.4,0.6,G0100010020400,2014-2018,Alabama,1,Autauga County,1,20400,"Tract 204, Autauga Alabama",3831,3831,3382,296,24,8,0,0,121,18,103,2769,14,151,744,175,508,272,602,177,74,52,1639,1578,31,16,13,9,8,3831.0,29.0,80.0,151.0,214.0,414.0,34.0,2909.0,54050.0,30527.0,3097.0,1752.0,1712.0,1663.0,49.0,40.0,1345.0,1867,1639,228,1639,1262,377,966.0,377.0,16.0,37.0,43.0,85.0,33.0,39.0,28.0,7.0,71.0,18.0,1639,1468,7,1315,146,171,3831,3720,0,30,30,51,0,0,0,0,0,0,145.0
4,1400000US01001020600,"Tract 206, Autauga",Autauga County,Alabama,South,66.2,71.9,5.7,G0100010020600,2014-2018,Alabama,1,Autauga County,1,20600,"Tract 206, Autauga Alabama",3705,3705,2726,878,0,0,0,57,44,0,44,2455,43,367,678,194,507,161,346,105,22,32,1332,1307,16,0,9,0,0,3689.0,100.0,696.0,330.0,187.0,308.0,38.0,2030.0,46688.0,28049.0,3004.0,1451.0,1443.0,1333.0,110.0,8.0,1553.0,1413,1332,81,1332,1031,301,732.0,301.0,7.0,79.0,54.0,8.0,1

## Regressions

### 2020 regressions

In [33]:
### 2020 Multi-regression

import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = regdf[['2010_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target20 = regdf[["2020_rate"]]

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
model20.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              2020_rate   R-squared (uncentered):                   0.991
Model:                            OLS   Adj. R-squared (uncentered):              0.991
Method:                 Least Squares   F-statistic:                          1.123e+05
Date:                Wed, 17 Jun 2020   Prob (F-statistic):                        0.00
Time:                        15:23:47   Log-Likelihood:                     -2.0178e+05
No. Observations:               62463   AIC:                                  4.037e+05
Df Residuals:                   62405   BIC:                                  4.042e+05
Df Model:                          58                                                  
Covariance Type:            nonrobust                                                  
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
2010_rate                        0.8197      0.002    441.744      0.000       0.816       0.823
total_population                -0.0011      0.003     -0.348      0.728      -0.007       0.005
white_alone                      0.0007      0.000      1.328      0.184      -0.000       0.002
black_alone                   1.458e-05      0.000      0.030      0.976      -0.001       0.001
amerindian_alone                -0.0042      0.001     -7.894      0.000      -0.005      -0.003
asian_alone                      0.0004      0.001      0.831      0.406      -0.001       0.001
pacific_islander_alone        7.365e-05      0.001      0.098      0.922      -0.001       0.002
other_alone                      0.0006      0.000      1.139      0.255      -0.000       0.002
two_or_more                      0.0013      0.000      3.315      0.001       0.001       0.002
two_more_including_other         0.0022      0.000      4.525      0.000       0.001       0.003
two_more_excluding_other        -0.0009      0.000     -2.658      0.008      -0.002      -0.000
total_education                  0.0014      0.000      9.315      0.000       0.001       0.002
no_school                        0.0027      0.001      4.515      0.000       0.002       0.004
some_school                     -0.0024      0.000    -10.396      0.000      -0.003      -0.002
diploma                         -0.0001      0.000     -0.708      0.479      -0.000       0.000
ged                             -0.0030      0.000     -8.142      0.000      -0.004      -0.002
some_college                     0.0041      0.000     22.216      0.000       0.004       0.004
associate                        0.0024      0.000      8.465      0.000       0.002       0.003
bachelor                         0.0001      0.000      0.629      0.529      -0.000       0.000
master                           0.0004      0.000      1.298      0.194      -0.000       0.001
prof_school                     -0.0058      0.001    -11.371      0.000      -0.007      -0.005
doctorate                        0.0030      0.001      4.957      0.000       0.002       0.004
language_total                   0.0003   7.95e-05      3.515      0.000       0.000       0.000
lang_english_only                0.0015      0.000      5.719      0.000       0.001       0.002
lang_spanish                    -0.0035      0.000    -12.613      0.000      -0.004      -0.003
lang_spanish_limited_english    -0.0015      0.001     -2.438      0.015      -0.003      -0.000
lang_other_indo_euro            -0.0035      0.000    -10.315      0.000      -0.004      -0.003
lang_asian_pacific_island       -0.0004      0.001     -0.711      0.477  

In [34]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

,variable,intercept,coefficient,r-squared
0,2010_rate,-12.534909,1.087104,0.727270
67,dial_up_computer,60.797335,0.072910,0.003468
21,doctorate,58.826686,0.067434,0.083398
20,prof_school,58.588280,0.049421,0.088341
74,us_born_abroad,59.840070,0.040010,0.016673
17,associate,53.305761,0.035124,0.123011
19,master,55.109618,0.027238,0.192682
27,lang_asian_pacific_island,60.389675,0.017690,0.019870
26,lang_other_indo_euro,60.148480,0.017616,0.016041
18,bachelor,52.633782,0.016930,0.233446


### Normalize inputs

In [35]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
norm_model20.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              2020_rate   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     4320.
Date:                Wed, 17 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:23:48   Log-Likelihood:            -2.0177e+05
No. Observations:               62463   AIC:                         4.037e+05
Df Residuals:                   62404   BIC:                         4.042e+05
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
2010_rate                       80.3795      0.333    241.190      0.000      79.726      81.033
total_population              -131.4928     58.413     -2.251      0.024    -245.983     -17.002
white_alone                     86.3385     70.277      1.229      0.219     -51.405     224.082
black_alone                     18.7571     19.825      0.946      0.344     -20.101      57.615
amerindian_alone               -12.5539     13.931     -0.901      0.368     -39.858      14.750
asian_alone                     21.2273     19.020      1.116      0.264     -16.051      58.506
pacific_islander_alone           5.4624      5.900      0.926      0.355      -6.102      17.026
other_alone                     19.9166     16.679      1.194      0.232     -12.775      52.608
two_or_more                      6.1880      4.746      1.304      0.192      -3.113      15.489
two_more_including_other         6.3362      1.942      3.262      0.001       2.530      10.143
two_more_excluding_other         3.7940      4.198      0.904      0.366      -4.434      12.022
total_education                -98.2655     17.066     -5.758      0.000    -131.715     -64.816
no_school                        8.4402      1.024      8.243      0.000       6.433      10.447
some_school                     16.7139      3.647      4.583      0.000       9.566      23.861
diploma                         23.6980      3.392      6.986      0.000      17.049      30.347
ged                              9.1032      2.432      3.744      0.000       4.337      13.869
some_college                    41.1367      3.645     11.286      0.000      33.993      48.281
associate                       15.3593      1.666      9.218      0.000      12.093      18.625
bachelor                        50.6964      7.046      7.195      0.000      36.887      64.506
master                          25.4421      3.509      7.250      0.000      18.564      32.320
prof_school                      1.8290      1.569      1.166      0.244      -1.246       4.904
doctorate                       13.3885      1.572      8.518      0.000      10.308      16.469
language_total                -100.5148     17.407     -5.774      0.000    -134.632     -66.397
lang_english_only              186.1348     27.881      6.676      0.000     131.488     240.782
lang_spanish                    56.7040     11.038      5.137      0.000      35.069      78.338
lang_spanish_limited_english    -2.0527      0.900     -2.280      0.023      -3.817      -0.288
lang_other_indo_euro            37.8791      7.376      5.135      0.000      23.422      52.337
lang_asian_pacific_island       54.8290      9.123      6.010      0.000      36.947      72.711
lang_other                      35.4366      4.466      7.935      0.000    

In [36]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

,variable,intercept,coefficient,r-squared
40,labor_force,50.593276,140.473246,0.123758
36,income_poverty_2_over,47.379214,129.989142,0.292712
18,bachelor,52.633782,122.253884,0.233446
72,us_born,50.466271,115.164558,0.107311
50,owner_occupied,48.147138,114.270232,0.267678
2,white_alone,50.684954,111.410310,0.169319
0,2010_rate,-12.534909,108.710430,0.727270
68,broadband_computer,48.287297,98.568239,0.192060
39,employment_total,51.421176,94.864375,0.084620
19,master,55.109618,94.677660,0.192682


### 2010 Regressions

In [37]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = regdf[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = regdf[["2010_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
model10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              2010_rate   R-squared (uncentered):                   0.963
Model:                            OLS   Adj. R-squared (uncentered):              0.963
Method:                 Least Squares   F-statistic:                          2.861e+04
Date:                Wed, 17 Jun 2020   Prob (F-statistic):                        0.00
Time:                        15:23:49   Log-Likelihood:                     -2.4981e+05
No. Observations:               62463   AIC:                                  4.997e+05
Df Residuals:                   62406   BIC:                                  5.002e+05
Df Model:                          57                                                  
Covariance Type:            nonrobust                                                  
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
total_population                 0.0132      0.007      1.899      0.058      -0.000       0.027
white_alone                      0.0035      0.001      3.282      0.001       0.001       0.006
black_alone                      0.0027      0.001      2.568      0.010       0.001       0.005
amerindian_alone                -0.0015      0.001     -1.326      0.185      -0.004       0.001
asian_alone                      0.0036      0.001      3.248      0.001       0.001       0.006
pacific_islander_alone           0.0026      0.002      1.622      0.105      -0.001       0.006
other_alone                      0.0037      0.001      3.472      0.001       0.002       0.006
two_or_more                     -0.0015      0.001     -1.744      0.081      -0.003       0.000
two_more_including_other        -0.0018      0.001     -1.696      0.090      -0.004       0.000
two_more_excluding_other         0.0003      0.001      0.415      0.678      -0.001       0.002
total_education                 -0.0064      0.000    -20.550      0.000      -0.007      -0.006
no_school                        0.0188      0.001     14.451      0.000       0.016       0.021
some_school                      0.0033      0.000      6.697      0.000       0.002       0.004
diploma                          0.0063      0.000     19.239      0.000       0.006       0.007
ged                              0.0012      0.001      1.461      0.144      -0.000       0.003
some_college                     0.0059      0.000     14.914      0.000       0.005       0.007
associate                        0.0138      0.001     22.947      0.000       0.013       0.015
bachelor                        -0.0004      0.000     -1.006      0.315      -0.001       0.000
master                          -0.0121      0.001    -19.144      0.000      -0.013      -0.011
prof_school                     -0.0557      0.001    -51.227      0.000      -0.058      -0.054
doctorate                        0.0125      0.001      9.673      0.000       0.010       0.015
language_total                   0.0079      0.000     46.579      0.000       0.008       0.008
lang_english_only                0.0059      0.001     10.691      0.000       0.005       0.007
lang_spanish                     0.0041      0.001      6.791      0.000       0.003       0.005
lang_spanish_limited_english     0.0014      0.001      1.013      0.311      -0.001       0.004
lang_other_indo_euro            -0.0078      0.001    -10.648      0.000      -0.009      -0.006
lang_asian_pacific_island        0.0011      0.001      0.918      0.358      -0.001       0.003
lang_other                       0.0046      0.001      3.319      0.001  

In [38]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

,variable,intercept,coefficient,r-squared
65,dial_up_computer,67.547845,0.053981,0.003089
20,doctorate,66.176595,0.047540,0.067352
19,prof_school,66.000524,0.034986,0.071941
72,us_born_abroad,67.047235,0.023943,0.009703
16,associate,62.920040,0.021971,0.078216
18,master,63.771388,0.018256,0.140654
26,lang_asian_pacific_island,67.292131,0.012207,0.015375
17,bachelor,62.185356,0.011204,0.166136
25,lang_other_indo_euro,67.192433,0.011141,0.010425
49,owner_occupied,58.877559,0.009532,0.205600


### Normalized 2010 Regressions

In [39]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
norm_model10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              2010_rate   R-squared:                       0.532
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     1247.
Date:                Wed, 17 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:23:50   Log-Likelihood:            -2.1322e+05
No. Observations:               62463   AIC:                         4.266e+05
Df Residuals:                   62405   BIC:                         4.271e+05
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
total_population             -4617.4387     67.644    -68.261      0.000   -4750.021   -4484.857
white_alone                   3665.2648     83.411     43.942      0.000    3501.780    3828.750
black_alone                   1027.1732     23.535     43.645      0.000     981.045    1073.302
amerindian_alone               689.8014     16.558     41.659      0.000     657.347     722.256
asian_alone                    980.5111     22.581     43.421      0.000     936.252    1024.771
pacific_islander_alone         291.6864      7.012     41.600      0.000     277.944     305.429
other_alone                    871.9438     19.795     44.049      0.000     833.146     910.742
two_or_more                    243.5835      5.635     43.227      0.000     232.539     254.628
two_more_including_other        86.6349      2.313     37.462      0.000      82.102      91.168
two_more_excluding_other       214.2706      4.986     42.979      0.000     204.499     224.042
total_education              -4139.9111     13.015   -318.086      0.000   -4165.421   -4114.402
no_school                      208.3136      0.935    222.718      0.000     206.480     210.147
some_school                    860.0172      2.895    297.062      0.000     854.343     865.692
diploma                        813.7777      2.629    309.574      0.000     808.625     818.930
ged                            545.1263      2.048    266.219      0.000     541.113     549.140
some_college                   868.2652      2.853    304.292      0.000     862.673     873.858
associate                      389.4506      1.335    291.771      0.000     386.834     392.067
bachelor                      1673.9837      5.534    302.493      0.000    1663.137    1684.830
master                         798.3051      2.909    274.394      0.000     792.603     804.007
prof_school                    310.6213      1.467    211.786      0.000     307.747     313.496
doctorate                      322.0770      1.428    225.536      0.000     319.278     324.876
language_total               -4151.0779     13.183   -314.887      0.000   -4176.916   -4125.240
lang_english_only             6813.2357     21.380    318.667      0.000    6771.330    6855.141
lang_spanish                  2690.8377      8.494    316.803      0.000    2674.190    2707.485
lang_spanish_limited_english     6.7678      1.081      6.260      0.000       4.649       8.887
lang_other_indo_euro          1783.4186      5.748    310.285      0.000    1772.153    1794.684
lang_asian_pacific_island     2189.1583      7.172    305.228      0.000    2175.101    2203.216
lang_other                    1061.9985      3.555    298.771      0.000    1055.032    1068.965
income_poverty_ratio         -2806.9543      8.817   -318.344      0.000   -

In [40]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

,variable,intercept,coefficient,r-squared
39,labor_force,60.590561,96.167608,0.094252
35,income_poverty_2_over,58.543424,87.560303,0.215818
17,bachelor,62.185356,80.905353,0.166136
1,white_alone,60.383297,79.103586,0.138706
49,owner_occupied,58.877559,78.562395,0.205600
70,us_born,60.609335,77.718157,0.079414
38,employment_total,60.964184,66.797547,0.068177
69,us_pop,61.200334,65.393300,0.062801
0,total_population,61.300946,65.293739,0.062799
20,doctorate,66.176595,63.798468,0.067352


## Edited Regressions
### Run regressions with fewer variables

In [41]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = regdf[['2010_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = regdf[["2020_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
model_ed.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              2020_rate   R-squared (uncentered):                   0.990
Model:                            OLS   Adj. R-squared (uncentered):              0.990
Method:                 Least Squares   F-statistic:                          1.803e+05
Date:                Wed, 17 Jun 2020   Prob (F-statistic):                        0.00
Time:                        15:23:51   Log-Likelihood:                     -2.0277e+05
No. Observations:               62463   AIC:                                  4.056e+05
Df Residuals:                   62428   BIC:                                  4.059e+05
Df Model:                          35                                                  
Covariance Type:            nonrobust                                                  
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
2010_rate                        0.8426      0.002    495.869      0.000       0.839       0.846
total_population                -0.0005      0.000     -1.525      0.127      -0.001       0.000
white_alone                  -7.794e-05   8.72e-05     -0.894      0.371      -0.000    9.29e-05
black_alone                     -0.0008   9.19e-05     -8.420      0.000      -0.001      -0.001
amerindian_alone                -0.0050      0.000    -20.262      0.000      -0.006      -0.005
asian_alone                      0.0005      0.000      3.658      0.000       0.000       0.001
pacific_islander_alone          -0.0009      0.001     -1.416      0.157      -0.002       0.000
no_school                        0.0033      0.001      4.889      0.000       0.002       0.005
some_school                     -0.0011      0.000     -4.862      0.000      -0.001      -0.001
diploma                          0.0014      0.000      8.780      0.000       0.001       0.002
ged                             -0.0005      0.000     -1.200      0.230      -0.001       0.000
some_college                     0.0060      0.000     32.751      0.000       0.006       0.006
associate                        0.0044      0.000     14.258      0.000       0.004       0.005
bachelor                         0.0015      0.000      8.004      0.000       0.001       0.002
master                           0.0032      0.000     11.121      0.000       0.003       0.004
lang_english_only                0.0045      0.000     12.020      0.000       0.004       0.005
lang_spanish                    -0.0008      0.000     -1.926      0.054      -0.002    1.36e-05
lang_spanish_limited_english -2.679e-05      0.001     -0.042      0.966      -0.001       0.001
lang_other                       0.0105      0.001     12.396      0.000       0.009       0.012
income_poverty_ratio             0.0009      0.000      7.812      0.000       0.001       0.001
per_capita_income             1.967e-05   2.87e-06      6.861      0.000    1.41e-05    2.53e-05
civilian_employed               -0.0003      0.000     -2.569      0.010      -0.001   -8.23e-05
civilian_unemployed             -0.0006      0.000     -1.485      0.138      -0.001       0.000
not_labor_force                 -0.0010      0.000     -7.243      0.000      -0.001      -0.001
total_houses                    -0.0036      0.000    -35.677      0.000      -0.004      -0.003
occupied_houses                 -0.0017      0.000     -8.770      0.000      -0.002      -0.001
vacant_houses                   -0.0019      0.000    -16.239      0.000      -0.002      -0.002
owner_occupied                   0.0002      0.000      1.523      0.128  

In [42]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

,variable,intercept,coefficient,r-squared
0,2010_rate,-12.534909,1.087104,0.727270
33,dial_up_computer,60.797335,0.072910,0.003468
12,associate,53.305761,0.035124,0.123011
14,master,55.109618,0.027238,0.192682
13,bachelor,52.633782,0.016930,0.233446
27,owner_occupied,48.147138,0.013864,0.267678
11,some_college,54.068113,0.012939,0.069428
34,broadband_computer,48.153947,0.011113,0.192060
29,median_gross_rent,50.834835,0.009623,0.115247
32,has_computer,48.993469,0.009315,0.144513
